# RAG进阶操作

In [1]:
!pip install langchain_chroma 

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
  Using cached https://mirrors.aliyun.com/pypi/packages/05/fd/36d8d37a257d7f5961cea9ef0e1a9616a4b4fc259cad2cbf02416de90215/langchain_chroma-1.0.0-py3-none-any.whl (12 kB)
  Using cached https://mirrors.aliyun.com/pypi/packages/59/91/bf5dfa3bd5b457a9195439e077128c274202b1a231bd3a9d4d7f3c2259a3/chromadb-1.3.6-cp39-abi3-win_amd64.whl (21.9 MB)
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---- ----------------------------------- 1.6/12.8 MB 11.1 MB/s eta 0:00:02
     ------------ --------------------------- 3.9/12.8 MB 11.3 MB/s eta 0:00:01
     ------------------- -------------------- 6.3/12.8 MB 11.5 MB/s eta 0:00:01
     --------------------------- ------------ 8.7/12.8 MB 11.6 MB/s eta 0:00:01
     ---------------------------------- ---- 11.3/12.8 MB 11.7 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 11.7 MB/s eta 0:00:01
     --------------------------------------  

In [2]:
import os
os.environ["DEEPSEEK_API_BASE"] = 'https://api.deepseek.com/v1'
os.environ["DEEPSEEK_API_KEY"] = 'sk-60e9910f323c46c3aeb645951b2d45ec'

In [4]:
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 13, 'total_tokens': 16, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 13}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '37a846d1-d38b-4235-81d6-468b2813aa64', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b0e38-dd7f-7672-b746-d05cceec9a17-0', usage_metadata={'input_tokens': 13, 'output_tokens': 3, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

### 读取文件夹内容

In [6]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import SpacyTextSplitter

### 指定文件夹路径 

In [7]:
directory_path = r'./RAG_data2'

### 创建一个DirectoryLoader对象

In [8]:
!pip install chardet

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
  Using cached https://mirrors.aliyun.com/pypi/packages/38/6f/f5fbc992a329ee4e0f288c1fe0e2ad9485ed064cac731ed2fe47dcc38cbf/chardet-5.2.0-py3-none-any.whl (199 kB)


In [9]:
text_loader_kwargs = {"autodetect_encoding": True}

#loader = DirectoryLoader(directory_path, glob="**/*.txt", loader_cls=lambda path: TextLoader(path, encoding="utf-8"), show_progress=True)
loader = DirectoryLoader(
    directory_path,
    glob="**/*.txt",
    loader_cls=TextLoader,  # NOT lambda, NOT encoding=utf-8
    show_progress=True,
    loader_kwargs=text_loader_kwargs
)

### 加载文件夹中的所有文件

In [10]:
documents = loader.load()

  0%|          | 0/11 [00:00<?, ?it/s]

100%|██████████| 11/11 [00:00<00:00, 31.94it/s]


In [11]:
print(len(documents))

11


In [12]:
from langchain_text_splitters import __all__
print(__all__)

['CharacterTextSplitter', 'ElementType', 'ExperimentalMarkdownSyntaxTextSplitter', 'HTMLHeaderTextSplitter', 'HTMLSectionSplitter', 'HTMLSemanticPreservingSplitter', 'HeaderType', 'JSFrameworkTextSplitter', 'KonlpyTextSplitter', 'Language', 'LatexTextSplitter', 'LineType', 'MarkdownHeaderTextSplitter', 'MarkdownTextSplitter', 'NLTKTextSplitter', 'PythonCodeTextSplitter', 'RecursiveCharacterTextSplitter', 'RecursiveJsonSplitter', 'SentenceTransformersTokenTextSplitter', 'SpacyTextSplitter', 'TextSplitter', 'TokenTextSplitter', 'Tokenizer', 'split_text_on_tokens']


### 使用文本分割器将文档分割成更小的部分

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100,separators=["\n", "!", "?", "。", "！", "？"],keep_separator=False )
splitted_docs = text_splitter.split_documents(documents)

In [14]:
print(len(splitted_docs))

133


In [15]:
print(splitted_docs)

[Document(metadata={'source': 'RAG_data2\\中华人民共和国个人信息保护法.txt'}, page_content='第一章\u3000总\u3000\u3000则\n第一条\u3000为了保护个人信息权益，规范个人信息处理活动，促进个人信息合理利用，根据宪法，制定本法。\n第二条\u3000自然人的个人信息受法律保护，任何组织、个人不得侵害自然人的个人信息权益。\n第三条\u3000在中华人民共和国境内处理自然人个人信息的活动，适用本法。\n在中华人民共和国境外处理中华人民共和国境内自然人个人信息的活动，有下列情形之一的，也适用本法：\n（一）以向境内自然人提供产品或者服务为目的；\n（二）分析、评估境内自然人的行为；\n（三）法律、行政法规规定的其他情形。\n第四条\u3000个人信息是以电子或者其他方式记录的与已识别或者可识别的自然人有关的各种信息，不包括匿名化处理后的信息。\n个人信息的处理包括个人信息的收集、存储、使用、加工、传输、提供、公开、删除等。\n第五条\u3000处理个人信息应当遵循合法、正当、必要和诚信原则，不得通过误导、欺诈、胁迫等方式处理个人信息。\n第六条\u3000处理个人信息应当具有明确、合理的目的，并应当与处理目的直接相关，采取对个人权益影响最小的方式。\n收集个人信息，应当限于实现处理目的的最小范围，不得过度收集个人信息。\n第七条\u3000处理个人信息应当遵循公开、透明原则，公开个人信息处理规则，明示处理的目的、方式和范围。\n第八条\u3000处理个人信息应当保证个人信息的质量，避免因个人信息不准确、不完整对个人权益造成不利影响。'), Document(metadata={'source': 'RAG_data2\\中华人民共和国个人信息保护法.txt'}, page_content='第七条\u3000处理个人信息应当遵循公开、透明原则，公开个人信息处理规则，明示处理的目的、方式和范围。\n第八条\u3000处理个人信息应当保证个人信息的质量，避免因个人信息不准确、不完整对个人权益造成不利影响。\n第九条\u3000个人信息处理者应当对其个人信息处理活动负责，并采取必要措施保障所处理的个人信息的安全。\n第十条\u3000任何组织、个人不得非法收集、使用、加工、传输他人个人信息，不得

In [16]:
os.environ["OPENAI_API_BASE"] = 'https://api.xty.app/v1'
os.environ["OPENAI_API_KEY"] = 'sk-9OiJkk755GLEpfNU41E93135B6634035B3F1A98e0dC1Cb60'

In [17]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    splitted_docs,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
)

In [18]:
all_docs = vectorstore.get()
print(all_docs.keys())  

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas'])


In [19]:
# 获取 Chroma 存储的所有文档
all_data = vectorstore.get(include=['documents', 'metadatas', 'embeddings'])

# 取前3条文本内容
for i in range(3):
    print(f"Text: {all_data['documents'][i]}")
    print(f"Metadata: {all_data['metadatas'][i]}")
    print(f"ids: {all_data['ids'][i]}")
    print(f"embeddings: {all_data['embeddings'][i]}")

Text: 第一章　总　　则
第一条　为了保护个人信息权益，规范个人信息处理活动，促进个人信息合理利用，根据宪法，制定本法。
第二条　自然人的个人信息受法律保护，任何组织、个人不得侵害自然人的个人信息权益。
第三条　在中华人民共和国境内处理自然人个人信息的活动，适用本法。
在中华人民共和国境外处理中华人民共和国境内自然人个人信息的活动，有下列情形之一的，也适用本法：
（一）以向境内自然人提供产品或者服务为目的；
（二）分析、评估境内自然人的行为；
（三）法律、行政法规规定的其他情形。
第四条　个人信息是以电子或者其他方式记录的与已识别或者可识别的自然人有关的各种信息，不包括匿名化处理后的信息。
个人信息的处理包括个人信息的收集、存储、使用、加工、传输、提供、公开、删除等。
第五条　处理个人信息应当遵循合法、正当、必要和诚信原则，不得通过误导、欺诈、胁迫等方式处理个人信息。
第六条　处理个人信息应当具有明确、合理的目的，并应当与处理目的直接相关，采取对个人权益影响最小的方式。
收集个人信息，应当限于实现处理目的的最小范围，不得过度收集个人信息。
第七条　处理个人信息应当遵循公开、透明原则，公开个人信息处理规则，明示处理的目的、方式和范围。
第八条　处理个人信息应当保证个人信息的质量，避免因个人信息不准确、不完整对个人权益造成不利影响。
Metadata: {'source': 'RAG_data2\\中华人民共和国个人信息保护法.txt'}
ids: 05d4c006-e8c6-41d3-a3e9-1709651489e4
embeddings: [ 0.02749268 -0.00049827  0.02846967 ...  0.00165967 -0.0011266
  0.00498757]
Text: 第七条　处理个人信息应当遵循公开、透明原则，公开个人信息处理规则，明示处理的目的、方式和范围。
第八条　处理个人信息应当保证个人信息的质量，避免因个人信息不准确、不完整对个人权益造成不利影响。
第九条　个人信息处理者应当对其个人信息处理活动负责，并采取必要措施保障所处理的个人信息的安全。
第十条　任何组织、个人不得非法收集、使用、加工、传输他人个人信息，不得非法买卖、提供或者公开他人个人信息；不得从事危害国家安全、公共利益的个人信息处理活动。
第十一条　国家建

In [20]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

# equal to below code

# def rag_chain(question):
#     context = retriever.invoke(question)
#     formatted_prompt = prompt.format(
#         question=question,
#         context=context,
#     )
#     return llm.invoke(formatted_prompt)

In [22]:
response = rag_chain.invoke("介绍一下互联网中如何保护儿童信息安全？")

print(response.content)

在互联网中保护儿童信息安全，以下措施可以被实施：

1. **法律法规**：根据《中华人民共和国网络安全法》《中华人民共和国未成年人保护法》等，制定明确的儿童个人信息保护规定。

2. **监护责任**：儿童监护人应正确履行监护职责，教育引导儿童增强个人信息保护意识和能力。

3. **行业自律**：鼓励互联网行业组织指导网络运营者制定儿童个人信息保护的行业规范，加强自律。

4. **原则遵循**：网络运营者在收集、存储、使用、转移、披露儿童信息时，要遵循正当必要、知情同意等原则。

5. **专门规则和协议**：网络运营者需设置专门的儿童个人信息保护规则和用户协议，并指定专人负责。

6. **告知监护人**：收集或使用儿童信息时，应以显著方式告知儿童监护人，并获取同意，提供拒绝选项。

7. **信息处理透明度**：明确告知收集、存储、使用信息的目的、存储期限、安全措施等。

8. **信息安全措施**：采取加密等技术措施保护信息安全。

9. **限制信息披露和转移**：未经法律允许或监护人同意，不得转移或披露儿童信息。

通过这些手段，可以有效保护儿童在网络中的个人信息安全。


### 1. 父文档检索器 （Parent Document Retriever）

In [ ]:
from langchain_classic.retrievers import ParentDocumentRetriever

In [ ]:
from langchain_core.stores import InMemoryStore

In [ ]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)

child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings()
)

store = InMemoryStore()

In [ ]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [28]:
retriever.add_documents(documents)

In [29]:
len(list(store.yield_keys()))

41

In [30]:
sub_docs = vectorstore.similarity_search("数据标注")

In [31]:
print(sub_docs[0].page_content)

（二）涉及知识产权的，不得侵害他人依法享有的知识产权；

（三）涉及个人信息的，应当取得个人同意或者符合法律、行政法规规定的其他情形；

（四）采取有效措施提高训练数据质量，增强训练数据的真实性、准确性、客观性、多样性；

（五）《中华人民共和国网络安全法》、《中华人民共和国数据安全法》、《中华人民共和国个人信息保护法》等法律、行政法规的其他有关规定和有关主管部门的相关监管要求。

第八条 在生成式人工智能技术研发过程中进行数据标注的，提供者应当制定符合本办法要求的清晰、具体、可操作的标注规则；开展数据标注质量评估，抽样核验标注内容的准确性；对标注人员进行必要培训，提升尊法守法意识，监督指导标注人员规范开展标注工作。

第三章 服务规范

第九条 提供者应当依法承担网络信息内容生产者责任，履行网络信息安全义务。涉及个人信息的，依法承担个人信息处理者责任，履行个人信息保护义务。


In [32]:
retrieved_docs = retriever.invoke("数据标注")

In [33]:
len(retrieved_docs[1].page_content)

1959

In [ ]:
print(retrieved_docs[0].page_content)

### 2. 多查询检索

In [34]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(documents)

# VectorDB
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

In [35]:
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = "如何保护用户的信息安全?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [ ]:
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [36]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain_classic.retrievers.multi_query").setLevel(logging.INFO)

In [37]:
unique_docs = retriever_from_llm.invoke(question)
len(unique_docs)

INFO:langchain_classic.retrievers.multi_query:Generated queries: ['1. 如何确保用户的个人信息不被泄露或滥用？', '2. 在数字平台中，如何加强用户数据的安全保护？', '3. 用户隐私安全的最佳防护措施有哪些？']


12

In [38]:
print(unique_docs[0].page_content)

第四十二条  网络运营者不得泄露、篡改、毁损其收集的个人信息；未经被收集者同意，不得向他人提供个人信息。但是，经过处理无法识别特定个人且不能复原的除外。

网络运营者应当采取技术措施和其他必要措施，确保其收集的个人信息安全，防止信息泄露、毁损、丢失。在发生或者可能发生个人信息泄露、毁损、丢失的情况时，应当立即采取补救措施，按照规定及时告知用户并向有关主管部门报告。

第四十三条  个人发现网络运营者违反法律、行政法规的规定或者双方的约定收集、使用其个人信息的，有权要求网络运营者删除其个人信息；发现网络运营者收集、存储的其个人信息有错误的，有权要求网络运营者予以更正。网络运营者应当采取措施予以删除或者更正。

第四十四条  任何个人和组织不得窃取或者以其他非法方式获取个人信息，不得非法出售或者非法向他人提供个人信息。

第四十五条  依法负有网络安全监督管理职责的部门及其工作人员，必须对在履行职责中知悉的个人信息、隐私和商业秘密严格保密，不得泄露、出售或者非法向他人提供。


In [ ]:
rag_chain = {"context": retriever_from_llm, "question": RunnablePassthrough()} | prompt | llm

In [ ]:
response = rag_chain.invoke("如何保护用户的信息安全?")

print(response.content)

### 3. Ensemble Retriever

### 安装 rank-bm25包

In [ ]:
!pip install rank-bm25 -i https://pypi.tuna.tsinghua.edu.cn/simple

In [39]:
from langchain_classic.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [40]:
bm25_retriever = BM25Retriever.from_documents(
    documents[0:6]
)

In [ ]:
pip install faiss-cpu -i https://pypi.tuna.tsinghua.edu.cn/simple

In [41]:
embedding = OpenAIEmbeddings()
faiss_vectorstore = FAISS.from_documents(
    documents[6:],embedding
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 4})

In [42]:
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [43]:
docs = ensemble_retriever.invoke("儿童")
docs

[Document(metadata={'source': 'RAG_data2\\互联网信息服务深度合成管理规定.txt'}, page_content='第一章 总则\n\n第一条 为了加强互联网信息服务深度合成管理，弘扬社会主义核心价值观，维护国家安全和社会公共利益，保护公民、法人和其他组织的合法权益，根据《中华人民共和国网络安全法》、《中华人民共和国数据安全法》、《中华人民共和国个人信息保护法》、《互联网信息服务管理办法》等法律、行政法规，制定本规定。\n\n第二条 在中华人民共和国境内应用深度合成技术提供互联网信息服务（以下简称深度合成服务），适用本规定。法律、行政法规另有规定的，依照其规定。\n\n第三条 国家网信部门负责统筹协调全国深度合成服务的治理和相关监督管理工作。国务院电信主管部门、公安部门依据各自职责负责深度合成服务的监督管理工作。\n\n地方网信部门负责统筹协调本行政区域内的深度合成服务的治理和相关监督管理工作。地方电信主管部门、公安部门依据各自职责负责本行政区域内的深度合成服务的监督管理工作。\n\n第四条 提供深度合成服务，应当遵守法律法规，尊重社会公德和伦理道德，坚持正确政治方向、舆论导向、价值取向，促进深度合成服务向上向善。\n\n第五条 鼓励相关行业组织加强行业自律，建立健全行业标准、行业准则和自律管理制度，督促指导深度合成服务提供者和技术支持者制定完善业务规范、依法开展业务和接受社会监督。\n\n第二章 一般规定\n\n第六条 任何组织和个人不得利用深度合成服务制作、复制、发布、传播法律、行政法规禁止的信息，不得利用深度合成服务从事危害国家安全和利益、损害国家形象、侵害社会公共利益、扰乱经济和社会秩序、侵犯他人合法权益等法律、行政法规禁止的活动。\n\n深度合成服务提供者和使用者不得利用深度合成服务制作、复制、发布、传播虚假新闻信息。转载基于深度合成服务制作发布的新闻信息的，应当依法转载互联网新闻信息稿源单位发布的新闻信息。\n\n第七条 深度合成服务提供者应当落实信息安全主体责任，建立健全用户注册、算法机制机理审核、科技伦理审查、信息发布审核、数据安全、个人信息保护、反电信网络诈骗、应急处置等管理制度，具有安全可控的技术保障措施。\n\n第八条 深度合成服务提供者应当制定和公开管理规则、平台公约，完善服务协议，依法依约履行管理责

In [44]:
len(docs)

8